# Sistema de recomendación canciones Spotify

In [168]:
# Importo librerias
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd

In [170]:
# Especifico credenciales API Spotify
cid = 'Client ID'
secret = 'Client secret'
username = 'Your user name (number profile)'
redirect_uri = 'http://yourhost/could_be_local_host'

# Defino el permiso para conectar con función 'current_user_top_tracks' de Spotify
# Tenemos otros metodos para los permisos de 'playlist-modify-private' o 'playlist-modify-public'
# sp.user_current_followed_artists
# sp.current_user_playlists
# sp.current_user_recently_played
# sp.current_user_top_artists
# Configuro los alcances que necesito
scopes = ["playlist-modify-private", "playlist-modify-public"]

# Crea una instancia de SpotifyOAuth con los alcances
sp_oauth = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scopes)

# Genero un token a traves de la funcion 'util' especificando las credenciales
token_info = sp_oauth.get_access_token()

# Creo una instancia de Spotify con el token de acceso
sp = spotipy.Spotify(auth=token_info["access_token"])

C:\Users\danal\AppData\Local\Temp\ipykernel_1948\1440825629.py:14: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token()


In [106]:
# Llamo al metodo 'current_user_top_tracks', limitando a las 20 canciones más recientes
# Top20 es un diccionario con información de las canciones y las keys 'items' (adjetivos) (ej: 'name':'David Bowie')
top20 = sp.current_user_top_tracks(time_range='short_term', limit=20)

# Itero el diccionario creado para imprimir una lista
for i, item in enumerate(top20['items']):
    print(i+1, item['name'], '//', item['artists'][0]['name'])

1 I Can't Stop - Original Mix // De-Javu
2 I Can't Stop - Main Mix // De-Javu
3 We Gonna Feel It - Radio Mix // Chris Lawyer
4 Tired of Being Alone // Al Green
5 Je Suis Le Ventre Du Monde // De-Javu
6 Old Time Lovin' // Al Green
7 I Can't Stop - Radio Edit // De-Javu
8 L-O-V-E (Love) // Al Green
9 Love and Happiness // Al Green
10 I've Never Found a Girl // Al Green
11 On Your Name // Dino MFU
12 I Can't Stop - Back To The 80's // De-Javu
13 Reckless // De-Javu
14 Here I Am (Come and Take Me) // Al Green
15 Remington Blues // De-Javu
16 High // Lighthouse Family
17 Whatever // Ohasis
18 Promises // The Cranberries
19 Lost & Found - Original Mix // 7 Skies
20 La Nouvelle Égérie // De-Javu


### Lista con canciones más escuchadas
Voy a crear un dataframe con los features, o caracteristicas de estas 20 canciones. Estas caracteristicas, me permitirán más adelante diferenciar una canción de otra y a la vez, encontrar similitudes con otras pistas que aún no haya escuchado y que podré incluir en una lista de reproducción de recomendaciones.

In [107]:
# Creo tres listas que contendrán toda la info de cada canción que quiero extraer
tracks = top20['items']
track_ids = []
track_names = []
features = []

# Itero mi lista top20 para extraer los datos que necesito
for track in tracks:
    track_id = track['id']
    track_name = track['name']
    #Aqui aplico la función 'sp.audio_features' para extraer las caracteristicas adicionales de cada canción.
    audio_features = sp.audio_features(track_id)
# Meto toda la info recolectada en las listas  
    track_ids.append(track_id)
    track_names.append(track_name)
    features.append(audio_features[0])

# Convierto el resultado en un dataframe
top20_df = pd.DataFrame(features,index = track_names)

In [108]:
top20_df.head(20)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
I Can't Stop - Original Mix,0.613,0.952,7,-7.706,1,0.0405,0.008650,0.347000,0.2960,0.870,127.004,audio_features,1wwYHkIRCFvzb4zzMiqOWD,spotify:track:1wwYHkIRCFvzb4zzMiqOWD,https://api.spotify.com/v1/tracks/1wwYHkIRCFvz...,https://api.spotify.com/v1/audio-analysis/1wwY...,266587,4
I Can't Stop - Main Mix,0.828,0.929,4,-8.614,0,0.0362,0.007770,0.182000,0.0748,0.664,127.990,audio_features,6l2pi16ncb5AKySaWaFfbs,spotify:track:6l2pi16ncb5AKySaWaFfbs,https://api.spotify.com/v1/tracks/6l2pi16ncb5A...,https://api.spotify.com/v1/audio-analysis/6l2p...,401120,4
We Gonna Feel It - Radio Mix,0.776,0.842,6,-9.506,1,0.0513,0.017500,0.846000,0.3420,0.789,129.975,audio_features,2js8YYMoUlFm1B5tl9lAWf,spotify:track:2js8YYMoUlFm1B5tl9lAWf,https://api.spotify.com/v1/tracks/2js8YYMoUlFm...,https://api.spotify.com/v1/audio-analysis/2js8...,196053,4
Tired of Being Alone,0.772,0.397,7,-8.585,1,0.0391,0.339000,0.005410,0.0753,0.617,97.964,audio_features,10vkYRKw6Jjr7try1ir50G,spotify:track:10vkYRKw6Jjr7try1ir50G,https://api.spotify.com/v1/tracks/10vkYRKw6Jjr...,https://api.spotify.com/v1/audio-analysis/10vk...,172320,4
Je Suis Le Ventre Du Monde,0.376,0.865,7,-2.313,1,0.0396,0.001220,0.000004,0.3810,0.478,96.014,audio_features,221r32Q3XgxGMVgn9zVYOJ,spotify:track:221r32Q3XgxGMVgn9zVYOJ,https://api.spotify.com/v1/tracks/221r32Q3XgxG...,https://api.spotify.com/v1/audio-analysis/221r...,178720,4
Old Time Lovin',0.658,0.481,9,-6.195,1,0.0586,0.669000,0.006400,0.1270,0.488,76.155,audio_features,3VvXwiETzZ7nGRk11n1mTp,spotify:track:3VvXwiETzZ7nGRk11n1mTp,https://api.spotify.com/v1/tracks/3VvXwiETzZ7n...,https://api.spotify.com/v1/audio-analysis/3VvX...,198529,4
I Can't Stop - Radio Edit,0.822,0.931,4,-8.245,0,0.0310,0.015000,0.007810,0.3460,0.822,127.997,audio_features,3pBSNPj4A0EQxtYDQMtSJQ,spotify:track:3pBSNPj4A0EQxtYDQMtSJQ,https://api.spotify.com/v1/tracks/3pBSNPj4A0EQ...,https://api.spotify.com/v1/audio-analysis/3pBS...,239187,4
L-O-V-E (Love),0.665,0.468,5,-9.558,1,0.0270,0.019000,0.000000,0.0987,0.699,109.432,audio_features,49qkJtDdi3KHmENFzleve4,spotify:track:49qkJtDdi3KHmENFzleve4,https://api.spotify.com/v1/tracks/49qkJtDdi3KH...,https://api.spotify.com/v1/audio-analysis/49qk...,189307,4
Love and Happiness,0.823,0.457,6,-10.961,1,0.2340,0.206000,0.000183,0.0763,0.735,99.163,audio_features,6SMHgPgNkhe9lneNTbgtel,spotify:track:6SMHgPgNkhe9lneNTbgtel,https://api.spotify.com/v1/tracks/6SMHgPgNkhe9...,https://api.spotify.com/v1/audio-analysis/6SMH...,307063,4
I've Never Found a Girl,0.828,0.443,0,-6.009,1,0.0292,0.357000,0.003820,0.2910,0.905,94.938,audio_features,6aTTKGTp7E5QjjDdq1ZoOV,spotify:track:6aTTKGTp7E5QjjDdq1ZoOV,https://api.spotify.com/v1/tracks/6aTTKGTp7E5Q...,https://api.spotify.com/v1/audio-analysis/6aTT...,220263,4


In [109]:
# Voy a reorganizar las columnas eliminando aquellas que no necesitaré, y dejando solo las que quiero
top20_df = top20_df[["id","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","valence"]]

## Construcción dataframe de canciones recomendadas
### Lista de artistas relacionados
Voy a crear un dataframe de canciones relacionadas, partiendo de la lista de las 20 canciones más escuchadas que acabamos de crear.

In [110]:
# Creo una lista donde se almacenarán los 20 artistas del top20
ids_artists = []
print('Artistas en mi top20:')
print('=====================')

# Itero la lista para extraer el id y el nombre del artista
for item in top20['items']:
    artist_id = item['artists'][0]['id']
    artist_name = item['artists'][0]['name']
    print(f'{artist_id}: {artist_name}')
    ids_artists.append(artist_id) # Almaceno resultados en la lista ids_artists

Artistas en mi top20:
6raaHJXd3aoB2F16ZAtLO0: De-Javu
6raaHJXd3aoB2F16ZAtLO0: De-Javu
7mjToYiXx9JeAXxWVG1kz0: Chris Lawyer
3dkbV4qihUeMsqN4vBGg93: Al Green
6raaHJXd3aoB2F16ZAtLO0: De-Javu
3dkbV4qihUeMsqN4vBGg93: Al Green
6raaHJXd3aoB2F16ZAtLO0: De-Javu
3dkbV4qihUeMsqN4vBGg93: Al Green
3dkbV4qihUeMsqN4vBGg93: Al Green
3dkbV4qihUeMsqN4vBGg93: Al Green
4V91yhFnYrVjpEoaRIDaiR: Dino MFU
6raaHJXd3aoB2F16ZAtLO0: De-Javu
6raaHJXd3aoB2F16ZAtLO0: De-Javu
3dkbV4qihUeMsqN4vBGg93: Al Green
6raaHJXd3aoB2F16ZAtLO0: De-Javu
6edGSAX5dVpeJVwu1Q0NwJ: Lighthouse Family
0tdUnfzh1I8uzI956s4hws: Ohasis
7t0rwkOPGlDPEhaOcVtOt9: The Cranberries
2mmqhYDTD0weseyXUf1QJ5: 7 Skies
6raaHJXd3aoB2F16ZAtLO0: De-Javu


In [111]:
# Para evitar repeticiones en esta lista, utilizo la función 'set'
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Número de artistas (sin repeticiones): 8


In [112]:
# Busco artistas relacionados con estos 8 que he seleccionado del top20
# Para ello utilizo el método 'artist_related_artists' de la API de Spotify, que nos devuelve artistas similares.
print('Artistas similares:')
print('=====================')
# Creo lista que contendrá los artistas relacionados
ids_similar_artists = []

# Itero la lista de artistas creada anteriormente, para pasarle al método los artistas de las IDs de las canciones
for artist_id in ids_artists:
    artists = sp.artist_related_artists(artist_id)['artists']
    # Para cada artista, devuélveme una lista con id y el nombre del artista
    for item in artists:
        artist_id = item['id']
        artist_name = item['name']
        print(f'{artist_id}: {artist_name}')
        ids_similar_artists.append(artist_id) # Almaceno en la lista ids_similar_artists

Artistas similares:
6ogn9necmbUdCppmNnGOdi: Alanis Morissette
1tqZaCwM57UFKjWoYwMLrw: The Cardigans
6S0GHTqz5sxK5f9HtLXn9q: Garbage
0lJlKQvuM2Sd9DPPyUXcHg: Sixpence None The Richer
0cQbJU1aAzvbEmTuljWLlF: No Doubt
0dlOr0VIysztGWvU1dpjmP: Natalie Imbruglia
2mpeljBig2IXLXRAFO9AAs: Dido
4nlqDmbzFzbNITaqjJv7D7: The Sundays
4TKTii6gnOnUXQHyuo9JaD: Sheryl Crow
1VbWUxZTRNY2gw3qZ1tg9W: The Corrs
0MBIKH9DjtBkv8O3nS6szj: 10,000 Maniacs
3g2kUQ6tHLLbmkV7T4GPtL: Fiona Apple
5SHQUMAmEK5KmuSb0aDvsn: Hole
4sD9znwiVFx9cgRPZ42aQ1: Sinéad O'Connor
5gznATMVO85ZcLTkE9ULU7: Lenny Kravitz
1KsASRNugxU85T0u6zSg32: Tori Amos
6kXp61QMZFPcKMcRPqoiVj: Gin Blossoms
40Yq4vzPs9VNUrIBG5Jr2i: The Smashing Pumpkins
0Je74SitssvJg1w4Ra2EK7: 4 Non Blondes
37w38cCSGgKLdayTRjna4W: Mazzy Star
1aRr1YCvofItUMebQWiT5c: Playmen
4V91yhFnYrVjpEoaRIDaiR: Dino MFU
2hkGkEnyudpE42IU4DBt99: Manuel Riva
1oiZoULxUJDYGOKMgVefP4: Morandi
3JEz4GnZ470OdepLILKQIR: Coyot
41ZhnBiaqeoe6asIzVDvqP: Serge Devant
6a46CV7CW5AgVBntbToHLH: TOMY or ZOX
4

In [113]:
# Añado el listado de artistas similares al listado inicial de artistas proveniente del top-20, con la función 'extend'
ids_artists.extend(ids_similar_artists)
# Elimino repeticiones de artistas como hice anteriormente con función 'set'
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Número de artistas (sin repeticiones): 159


### Ampliación lista con artistas de nuevos lanzamientos
Voy a ampliar el dataset de recomendaciones con nuevos lanzamientos, para tener la lista siempre actualizada.

In [114]:
# Voy a imprimir una lista de los artistas de los nuevos lanzamientos
print('')
print('Artistas con nuevos lanzamientos:')
print('=====================')
# Para agregar nuevos lanzamientos, utilizaré el método 'get_new_releases', y nos quedaremos con los albums, limitando a 20
new_releases = sp.new_releases(limit=20)['albums']
# De cada album en new_releases, extraeré el id y el nombre de los artistas, nombre del album y fecha de lanzamiento
for item in new_releases['items']:
    artist_id = item['artists'][0]['id'] #[0] porque puede haber varios artistas, tomo el primero
    artist_name = item['artists'][0]['name']
    album_name = item['name']   
    release_date = item['release_date']
    # Imprimo en pantalla el resultado de la iteración
    print(f'{artist_id}: {artist_name} - // {album_name}, {release_date}')
    # Añado la lista que crea la iteración artist_id a la lista principal top20 ids_artists
    ids_artists.append(artist_id) 


Artistas con nuevos lanzamientos:
1Mw40k757jZuiL0NIJpdO5: GULEED - // Cuando Menos Lo Espera, 2023-07-07
6k8mwkKJKKjBILo7ypBspl: Ana Mena - // bellodrama, 2023-03-24
7iK8PXO48WeuP03g8YR51W: Myke Towers - // LA VIDA ES UNA, 2023-03-23
5XJDexmWFLWOkjOEjOVX3e: Eladio Carrion - // 3MEN2 KBRN, 2023-03-17
790FomKkXshlbRYZFtlgla: KAROL G - // MAÑANA SERÁ BONITO, 2023-02-24
2auC28zjQyVTsiZKNgPRGs: RM - // Indigo, 2022-12-02
6KImCVD70vtIoJWnq6nGn3: Harry Styles - // Harry's House, 2022-05-20
4q3ewBCX7sLwd24euuV69X: Bad Bunny - // Un Verano Sin Ti, 2022-05-06
2R21vXR83lH98kGeO99Y66: Anuel AA - // Las Leyendas Nunca Mueren, 2021-11-26
4dpARuHxo51G3z768sgnrY: Adele - // 30, 2021-11-19
6eUKZXaKkcviH0Ku9w2n3V: Ed Sheeran - // =, 2021-10-29
4gzpq5DPGxSnKTe4SA8HAU: Coldplay - // Music Of The Spheres, 2021-10-15
53KwLdlmrlCelAZMaLVZqU: James Blake - // Friends That Break Your Heart, 2021-10-08
4MzJMcHQBl9SIYSjwWn8QW: Spiritbox - // Eternal Blue, 2021-09-17
1vyhD5VmyZ7KMfW5gqLgo5: J Balvin - // JOSE, 2

In [115]:
# Elimino repeticiones de artistas como hice anteriormente con función 'set'
ids_artists = list(set(ids_artists))
print(f'Número de artistas (sin repeticiones): {len(ids_artists)}')

Número de artistas (sin repeticiones): 179


### Selección de una album por artista
Como tenemos muchos artistas, para reducir el numero de solicitudes a la API, voy a limitar a un album por artista

In [116]:
# Para limitar a un album por artista, utilizaré la función 'sp.artist_albums'
# Creo lista donde almacenaré un album de cada artista
id_albums = []
# Creo una lista con el total de artistas para hacer de contador
nartists = len(ids_artists)
# Para cada artista recoge un album y lo mete en 'albums'
for i, id_artist in enumerate(ids_artists):
    print(f'Procesando artista {i+1} de {nartists}...')
    albums = sp.artist_albums(id_artist, limit=1) # Limitación a 1 album
    # Para cada album seleccionado extrae los 'items' y mete el id de cada album
    # en la lista de albums que hemos creado 'id_albums'
    for album in albums['items']:
        id_albums.append(album['id'])
print('Listo!')

Procesando artista 1 de 179...
Procesando artista 2 de 179...
Procesando artista 3 de 179...
Procesando artista 4 de 179...
Procesando artista 5 de 179...
Procesando artista 6 de 179...
Procesando artista 7 de 179...
Procesando artista 8 de 179...
Procesando artista 9 de 179...
Procesando artista 10 de 179...
Procesando artista 11 de 179...
Procesando artista 12 de 179...
Procesando artista 13 de 179...
Procesando artista 14 de 179...
Procesando artista 15 de 179...
Procesando artista 16 de 179...
Procesando artista 17 de 179...
Procesando artista 18 de 179...
Procesando artista 19 de 179...
Procesando artista 20 de 179...
Procesando artista 21 de 179...
Procesando artista 22 de 179...
Procesando artista 23 de 179...
Procesando artista 24 de 179...
Procesando artista 25 de 179...
Procesando artista 26 de 179...
Procesando artista 27 de 179...
Procesando artista 28 de 179...
Procesando artista 29 de 179...
Procesando artista 30 de 179...
Procesando artista 31 de 179...
Procesando artist

### Selección de tres canciones por album

In [53]:
# Para extraer tres canciones por album utilizaré la función 'sp.album_tracks'
# Creo lista donde almacenaré las tres canciones de cada album
id_tracks = []
# Creo una lista con el total de albums para hacer de contador
nalbums = len(id_albums)
# Para cada album recoge tres canciones y lo mete en 'album_tracks'
for i, id_album in enumerate(id_albums):
    print(f'Procesando album {i+1} de {nalbums}...')
    album_tracks = sp.album_tracks(id_album, limit=3)
    # Para cada cancion seleccionada extrae los 'items', y mete el id de cada canción que está en los items
    # en la lista que hemos creado 'id_tracks'
    for track in album_tracks['items']:
        id_tracks.append(track['id'])
        
print(f'Listo! Número total de canciones: {len(id_tracks)}')

Procesando album 1 de 179...
Procesando album 2 de 179...
Procesando album 3 de 179...
Procesando album 4 de 179...
Procesando album 5 de 179...
Procesando album 6 de 179...
Procesando album 7 de 179...
Procesando album 8 de 179...
Procesando album 9 de 179...
Procesando album 10 de 179...
Procesando album 11 de 179...
Procesando album 12 de 179...
Procesando album 13 de 179...
Procesando album 14 de 179...
Procesando album 15 de 179...
Procesando album 16 de 179...
Procesando album 17 de 179...
Procesando album 18 de 179...
Procesando album 19 de 179...
Procesando album 20 de 179...
Procesando album 21 de 179...
Procesando album 22 de 179...
Procesando album 23 de 179...
Procesando album 24 de 179...
Procesando album 25 de 179...
Procesando album 26 de 179...
Procesando album 27 de 179...
Procesando album 28 de 179...
Procesando album 29 de 179...
Procesando album 30 de 179...
Procesando album 31 de 179...
Procesando album 32 de 179...
Procesando album 33 de 179...
Procesando album 34

In [117]:
# Construyo el dataframe de canciones recomendadas, con los id que ya los tengo en (id_tracks), los nombres y los 13 features que
# habiamos seleccionado, extrayendo el nombre de cada canción con la función 'sp.track', y los features con la función 'sp.audio_features'

# Creo lista donde se almacenarán los nombres de las canciones
track_names = []
# Creo otra lista donde se almacenarán los features de las canciones
features = []
# Creo otra lista con la cantidad de canciones solo para hacer de contador
ntracks = len(id_tracks)
# Para cada id extrae el nombre y lo metes en 'track_name', y los features y los metes en 'audio_features'
for i, track_id in enumerate(id_tracks):
    print(f'Procesando track {i+1} de {ntracks}...')
    track_name = sp.track(track_id)['name']
    audio_features = sp.audio_features(track_id)
    # No incluir pistas sin "features", pues a veces las novedades aun no tienen features (caracteristicas)
    if audio_features[0] != None:
        track_names.append(track_name) # Añade los nombres a la lista 'track_names'
        features.append(audio_features[0]) # Añade los features a la lista 'audio_features'
print('¡Listo!')
# Creo dataframe con los features, y agrego los nombres de las canciones al index del dataframe 
recomendaciones_df = pd.DataFrame(features,index = track_names)

Procesando track 1 de 484...
Procesando track 2 de 484...
Procesando track 3 de 484...
Procesando track 4 de 484...
Procesando track 5 de 484...
Procesando track 6 de 484...
Procesando track 7 de 484...
Procesando track 8 de 484...
Procesando track 9 de 484...
Procesando track 10 de 484...
Procesando track 11 de 484...
Procesando track 12 de 484...
Procesando track 13 de 484...
Procesando track 14 de 484...
Procesando track 15 de 484...
Procesando track 16 de 484...
Procesando track 17 de 484...
Procesando track 18 de 484...
Procesando track 19 de 484...
Procesando track 20 de 484...
Procesando track 21 de 484...
Procesando track 22 de 484...
Procesando track 23 de 484...
Procesando track 24 de 484...
Procesando track 25 de 484...
Procesando track 26 de 484...
Procesando track 27 de 484...
Procesando track 28 de 484...
Procesando track 29 de 484...
Procesando track 30 de 484...
Procesando track 31 de 484...
Procesando track 32 de 484...
Procesando track 33 de 484...
Procesando track 34

Procesando track 269 de 484...
Procesando track 270 de 484...
Procesando track 271 de 484...
Procesando track 272 de 484...
Procesando track 273 de 484...
Procesando track 274 de 484...
Procesando track 275 de 484...
Procesando track 276 de 484...
Procesando track 277 de 484...
Procesando track 278 de 484...
Procesando track 279 de 484...
Procesando track 280 de 484...
Procesando track 281 de 484...
Procesando track 282 de 484...
Procesando track 283 de 484...
Procesando track 284 de 484...
Procesando track 285 de 484...
Procesando track 286 de 484...
Procesando track 287 de 484...
Procesando track 288 de 484...
Procesando track 289 de 484...
Procesando track 290 de 484...
Procesando track 291 de 484...
Procesando track 292 de 484...
Procesando track 293 de 484...
Procesando track 294 de 484...
Procesando track 295 de 484...
Procesando track 296 de 484...
Procesando track 297 de 484...
Procesando track 298 de 484...
Procesando track 299 de 484...
Procesando track 300 de 484...
Procesan

In [118]:
recomendaciones_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
Ring Ring,0.878,0.814,11,-5.871,0,0.0736,0.000478,0.746000,0.0913,0.3300,125.013,audio_features,0gIrlztrFenVS67A918ZXR,spotify:track:0gIrlztrFenVS67A918ZXR,https://api.spotify.com/v1/tracks/0gIrlztrFenV...,https://api.spotify.com/v1/audio-analysis/0gIr...,345605,4
Ring Ring - Radio Edit,0.863,0.803,11,-6.139,0,0.0611,0.001740,0.337000,0.2380,0.2740,125.005,audio_features,0stNFbUIH48OXmcqrDB9fJ,spotify:track:0stNFbUIH48OXmcqrDB9fJ,https://api.spotify.com/v1/tracks/0stNFbUIH48O...,https://api.spotify.com/v1/audio-analysis/0stN...,178565,4
Take Me Away (feat. Corbin),0.630,0.596,5,-5.476,0,0.0602,0.473000,0.000176,0.1490,0.4110,86.898,audio_features,1mb6mxgGHf5DuOHnJmvYFw,spotify:track:1mb6mxgGHf5DuOHnJmvYFw,https://api.spotify.com/v1/tracks/1mb6mxgGHf5D...,https://api.spotify.com/v1/audio-analysis/1mb6...,201133,4
Last Days,0.441,0.823,6,-5.964,0,0.0796,0.012700,0.001890,0.1640,0.2990,183.905,audio_features,2UU1YyRuo7bZTwi6pJHqPW,spotify:track:2UU1YyRuo7bZTwi6pJHqPW,https://api.spotify.com/v1/tracks/2UU1YyRuo7bZ...,https://api.spotify.com/v1/audio-analysis/2UU1...,154040,4
Thy Motion,0.632,0.790,5,-7.367,0,0.0981,0.153000,0.000196,0.2470,0.6300,168.076,audio_features,6hUti0aEC1PjqSmD29Pkv4,spotify:track:6hUti0aEC1PjqSmD29Pkv4,https://api.spotify.com/v1/tracks/6hUti0aEC1Pj...,https://api.spotify.com/v1/audio-analysis/6hUt...,148693,4
Scared,0.744,0.785,9,-8.087,0,0.0894,0.178000,0.000186,0.1620,0.2380,110.104,audio_features,62fe2qqQ6tlkCKgEU3thDs,spotify:track:62fe2qqQ6tlkCKgEU3thDs,https://api.spotify.com/v1/tracks/62fe2qqQ6tlk...,https://api.spotify.com/v1/audio-analysis/62fe...,166364,4
Syro,0.774,0.822,4,-10.044,0,0.0614,0.008450,0.901000,0.0970,0.0513,118.005,audio_features,09dYwR40hH1f1WMLAKNRv6,spotify:track:09dYwR40hH1f1WMLAKNRv6,https://api.spotify.com/v1/tracks/09dYwR40hH1f...,https://api.spotify.com/v1/audio-analysis/09dY...,406488,4
Pow You're in Love,0.645,0.570,5,-4.922,1,0.0685,0.746000,0.000000,0.0547,0.9200,111.923,audio_features,0UX2wx3YkpCV9AwGU4jLhs,spotify:track:0UX2wx3YkpCV9AwGU4jLhs,https://api.spotify.com/v1/tracks/0UX2wx3YkpCV...,https://api.spotify.com/v1/audio-analysis/0UX2...,133880,4
Billy the Kid,0.560,0.777,4,-3.577,1,0.0461,0.702000,0.000000,0.3300,0.8180,125.966,audio_features,1aEADQhyn8mhrCe1YJ0Uk7,spotify:track:1aEADQhyn8mhrCe1YJ0Uk7,https://api.spotify.com/v1/tracks/1aEADQhyn8mh...,https://api.spotify.com/v1/audio-analysis/1aEA...,128253,4
You're on My Mind,0.555,0.548,4,-5.057,1,0.0297,0.797000,0.000000,0.3380,0.9310,124.172,audio_features,4rczSNAU6dTmK5heEG5yDy,spotify:track:4rczSNAU6dTmK5heEG5yDy,https://api.spotify.com/v1/tracks/4rczSNAU6dTm...,https://api.spotify.com/v1/audio-analysis/4rcz...,159133,4


In [119]:
# Ahora tengo dos listas la top20_df, y la de recomendaciones_df que he creado, que voy a limpiar, eliminando las columnas
# o variables que no me interesan y dejando las que quiero igual que había hecho con la lista de top20_df
recomendaciones_df = recomendaciones_df[["id","acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","speechiness","tempo","valence"]]

### Comparación de las dos listas
Para hacer las comparativas, haré un filtrado basado en el contenido, utilizando la similitud del coseno, tomando las 12 características que tengo de cada canción y comparándolas entre ellas, para quedarme de entre todas solo con las canciones más similares a las más escuchadas 'top20_df'. Transformaré cada canción en un array o vector de doce valores (solo de los features), para poder comparar las similitudes de las características entre sí calculando un producto, que luego dividiré entre el número de dimensiones del vector, y así de esta manera extraer el coseno del ángulo.
Ésta similitud variará entre dos valores numéricos (-1 y 1), cuanto más simil sea más cercano se encontrará del 1, y cuanto menos simil sea, mas cercano estará del 0 o -1.

In [120]:
# Extraigo solo los features en formato array, de las dos listas 'top20_df' y 'recomendaciones_df'
top20_mtx = top20_df.iloc[:,1:].values # A partir de la posición 1, pues la posición cero es el id
recomendaciones_mtx = recomendaciones_df.iloc[:,1:].values

In [121]:
# Normalizo todas las escalas de los valores de las características de las dos listas, a mu=0 sigma=1, utilizando la función 
# 'StandardScaler' de sklearn, restando el promedio a cada valor y dividiendo el resultado por la desviación standar.
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
recom_scaled = scaler.fit_transform(recomendaciones_mtx)
# Compruebo que tengo todos los datos normalizados o escalados, con los promedios y las desviaciones standar
print(t20_scaled.mean(axis=0))
print(t20_scaled.std(axis=0))

[ 3.88578059e-17  5.38458167e-16 -1.05471187e-16  9.99200722e-17
  4.99600361e-17 -1.11022302e-17 -1.66533454e-17 -6.66133815e-17
  8.88178420e-17 -2.19269047e-16 -7.93809463e-16  2.19269047e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [122]:
# Normalizo por filas, los tamaños de los vectores, para que todos sean iguales en tamaño.

# Saco las magnitudes de cada vector de las dos listas, elevando cada valor al cuadrado, sumando los resultados entre ellos
# y al resultado le saco la raiz cuadrada
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
recom_norm = np.sqrt((can_scaled*recom_scaled).sum(axis=1))

# Normalizo cada uno de los vectores de las dos listas, tomando sus valores y dividiéndolos por su magnitud
nt20 = t20_scaled.shape[0]
nrec = recom_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
recom = recom_scaled/recom_norm.reshape(ncan,1)

# Ahora ya tengo las listas de las características ya normalizadas en 't20' y 'recom'
# Compruebo que las magnitudes sean iguales a 1
print(np.sqrt((t20*t20).sum(axis=1)))
print(np.sqrt((recom*recom).sum(axis=1)))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [123]:
# Calculo las similitudes del coseno entre las dos listas con la función 'linear_kernel' de sklearn
# Ahora tengo una lista con 20 filas (top20), comparadas con las 484 canciones (recomendaciones) en las columnas, con un valor de similitud.
cos_sim = linear_kernel(t20,recom)
cos_sim.shape

(20, 484)

In [124]:
# Ahora voy a seleccionar todas aquellas canciones que esten mas cercanas a la similitud 1 que son las que mas se parecen a las 
# canciones del top20. Para ello voy a crear una función para extraer de la lista de similitudes, aquellas canciones que sean 
# iguales o mayores que una similitud de 0,7
# Para cada canción del top20, extrae un número determinado de recomendaciones (nrecom), de la lista de similitudes 'cos_sim',
# con un umbral.
def obtener_recomendaciones(pos, cos_sim, nrecom, umbral = 0.7):
    # Obtengo todas las canciones recomendadas por encima de umbral, y las almaceno en 'idx'
    idx = np.where(cos_sim[pos,:]>=umbral)[0]
    
    # Organizo las canciones similares almacenadas en 'idx', de forma descendente de mayor a menor, 
    # ya que me interesa que las primeras canciones sean las que más se parezcan.
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]] # [::-1] pues por defecto 'argsort' organiza de manera ascendente

    # Si hay más de "nrecom", retornar máximo los numeros de "nrecom" elegidos.
    if len(idx) >= nrecom:
        recoms = idx[0:nrecom]
    else:
        recoms = idx
  
    return recoms

In [125]:
# Voy a utilizar la función, para seleccionar las cinco canciones más similares de cada canción del top20
# Para cada una de las 20 canciones, recorre la lista de similitudes 'cos_sim' desde la posición de inicio,
# y elige cinco, con el umbral de la función.
for i in range(20):
    recoms = obtener_recomendaciones(i, cos_sim, 5)
    # Imprimo la canción primera, con las 5 recomendaciones y sus similitudes
    print(f'{i}==> pistas filtradas: {recoms}, similitudes: {cos_sim[i,recoms]}')

0==> pistas filtradas: [ 10 364 311  61 110], similitudes: [0.82774285 0.79172604 0.7517118  0.73039466 0.7238132 ]
1==> pistas filtradas: [ 19 136 247 295 398], similitudes: [0.83338109 0.80837373 0.76358477 0.75897557 0.74962788]
2==> pistas filtradas: [], similitudes: []
3==> pistas filtradas: [144 410 154 294 200], similitudes: [0.87100839 0.86348144 0.84628941 0.84387001 0.82616025]
4==> pistas filtradas: [466 418 419 386 236], similitudes: [0.95476142 0.87106414 0.86685538 0.81407264 0.788397  ]
5==> pistas filtradas: [294 353 161 144  90], similitudes: [0.89263029 0.8291033  0.79898643 0.79742246 0.7786836 ]
6==> pistas filtradas: [121 282], similitudes: [0.77287832 0.72827184]
7==> pistas filtradas: [318 199 317 339 200], similitudes: [0.79265172 0.79043223 0.78769891 0.76120333 0.74693529]
8==> pistas filtradas: [469 411 351 362 449], similitudes: [0.92272    0.87025169 0.83052603 0.78687006 0.77930774]
9==> pistas filtradas: [293 472   9  94  95], similitudes: [0.77732307 0.7

Se puede observar como hay canciones que no tienen ninguna canción con un umbral de 0,8 o superior de similitud, y hay canciones que solo tienen tres y otras las cinco que hemos solicitado a la función

### Creación de playlist para Spotify

In [126]:
# Para crear la playlist solo me harán falta los IDs de las canciones, y para ello voy a almacenar los IDs en dos listas
# una para las top20 y otra para la playlist final que pasaré a la IPI de Spotify
ids_t20 = []
ids_playlist = []
# Para cada canción de la lista 'top20_df' imprime...
for i in range(top20_df.shape[0]):
    print(top20_df.index[i])   # el nombre de la pista que se analiza
    ids_t20.append(top20_df['id'][i]) # y extrae el id y lo almacena en lista de IDs 'ids_t20'
    
    # Llamo a la función de nuevo para obtener el listado de recomendadas para cada canción
    recoms = obtener_recomendaciones(i, cos_sim, 5, umbral=0.7)
    
    # Si no hay canciones recomendadas por encima del umbral 0,7 para la canción analizada, imprimo en pantalla
    if len(recoms)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        # Pero si, sí que hay canciones recomendadas por encima del umbral de la canción analizada, obten los ids 
        # correspondientes, almacénalos en la lista 'ids_playlist', y luego imprime los nombres
        for j in recoms:
            id_recom = recomendaciones_df['id'][j]
            ids_playlist.append(id_recom)
            print(f'   {recomendaciones_df.index[j]}')

I Can't Stop - Original Mix
   Feel The Beat
   Moving On
   It Only Takes A Minute - Odyssey Mix
   Are You Ready - Live at the Fillmore East, New York, NY [Show 1] - October 4, 1968
   Dos Besos Después
I Can't Stop - Main Mix
   Love Games - U.S. Remix
   Coraçao - Axwell Remix
   No Turning Back - Original Mix
   There Goes My Heart - Again
   Dirty Funk
We Gonna Feel It - Radio Mix
     ***No se encontraron pistas relacionadas***
Tired of Being Alone
   Power to the People - Demo Version
   Bulletproof Love
   Stay With Me (By The Sea)
   California
   12:00 am ballad
Je Suis Le Ventre Du Monde
   Je Suis Le Ventre Du Monde
   Hurt You
   Yellowjacket (feat. Sam Carter)
   My Dear Machine
   See The Light
Old Time Lovin'
   California
   Moon
   Strangers By Nature
   Power to the People - Demo Version
   Why Fool Yourself
I Can't Stop - Radio Edit
   Rayos de Sol
   Zodiac
L-O-V-E (Love)
   Operator
   Diary of a smoker
   Letter Full of Tears
   My Broken Souvenirs
   12:00 am b

In [127]:
# Siempre es posible que algunas de las recomendaciones sean las mismas canciones que están en el top20, 
# por lo tanto es recomendable depurar la lista

# Elimino las IDs de las recomendaciones que ya están en la lista 'ids_top20'
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]

# Elimino posibles repeticiones en la lista entera con la función 'set', y lo almaceno en una nueva lista depurada
ids_playlist_dep = list(set(ids_playlist_dep))

In [128]:
len(ids_playlist_dep) # Al final de 500 canciones recomendadas, he obtenido una playlist de 77 recomendaciones

77

In [171]:
# Creo la playlist utilizando la función 'sp.user_playlist_create'
pl = sp.user_playlist_create(user = username, 
                            name = 'Playlist de recomendaciones con Spotipy',
                            description = 'Playlist creada con el sistema de recomendación')

In [172]:
# Añado a la playlist que he creado 'pl' las IDs seleccionadas de la lista 'ids_playlist_dep',
# utilizando la función 'sp.playlist_add_items' de la API de Spotify
sp.playlist_add_items(pl['id'],ids_playlist_dep)

{'snapshot_id': 'Myw0MDFmYTkyNDRiMDI2OGJhM2NmNzJiYWJlMjFkYWQwZjgxMTM4YzBj'}